# PyTorch quickstart tutorial 
This is the pytorch beginners tutorial found [here](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html). 

In [1]:
import torch 
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import ToTensor 

In [2]:
# Download training data from open datasets 
training_data = datasets.FashionMNIST(
    root = "data", 
    train = True, 
    download = True,
    transform = ToTensor()
)
# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root = "data", 
    train = False, 
    download = True, 
    transform = ToTensor()
)

In [3]:
# The size of batch, each element in the dataloader iterable wll return a batch of 64 features and labels 
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break 

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating the Model 

To define a neural network we:
* create a class that inherits from nn.Module. 
* define layers in the __init__ function. 
* specify how data will pass through the network in the forward function 

In [4]:
# Get cpu or gpu device for training 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [5]:
# Define model 
class NeuralNetwork(nn.Module):
    def __init__(self): 
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(), 
            nn.Linear(512, 512), 
            nn.ReLU(), 
            nn.Linear(512, 10)
        )

    def forward(self, x): 
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits 

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# Optimising the model parameters 

loss_fn = nn.CrossEntropyLoss() # The loss function 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # Optimiser function 

def train(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y =X.to(device), y.to(device)

        # Compute prediction error 
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0: # note: % 100 == 0 checks if the batch is divisble by 100 
            loss, current = loss.item(), batch *len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]") # :> is to do with alignment - so table looks nice

# Can check the model's performance against the test dataset 
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0 
    with torch.no_grad():
        for X, y in dataloader: 
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct) :>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Training is conducted over several epochs 
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("All done!")

Epoch 1
--------------------------
loss: 2.302437 [    0/60000]
loss: 2.288418 [ 6400/60000]
loss: 2.266029 [12800/60000]
loss: 2.260637 [19200/60000]
loss: 2.238847 [25600/60000]
loss: 2.207218 [32000/60000]
loss: 2.220316 [38400/60000]
loss: 2.182353 [44800/60000]
loss: 2.181254 [51200/60000]
loss: 2.140085 [57600/60000]
Test Error: 
 Accuracy: 35.9%, Avg loss: 2.138065 

Epoch 2
--------------------------
loss: 2.149500 [    0/60000]
loss: 2.136883 [ 6400/60000]
loss: 2.072231 [12800/60000]
loss: 2.092530 [19200/60000]
loss: 2.023451 [25600/60000]
loss: 1.967580 [32000/60000]
loss: 1.996394 [38400/60000]
loss: 1.913218 [44800/60000]
loss: 1.924055 [51200/60000]
loss: 1.837908 [57600/60000]
Test Error: 
 Accuracy: 56.4%, Avg loss: 1.841578 

Epoch 3
--------------------------
loss: 1.879369 [    0/60000]
loss: 1.841132 [ 6400/60000]
loss: 1.722915 [12800/60000]
loss: 1.768073 [19200/60000]
loss: 1.640613 [25600/60000]
loss: 1.611942 [32000/60000]
loss: 1.629708 [38400/60000]
loss: 1.

In [7]:
# Saving the model 
# A common way to save the model is to serialise the internal state dictionary 

torch.save(model.state_dict(), "model.pth")
print(" Saved pyTorch Model State to model.pth")

 Saved pyTorch Model State to model.pth


In [8]:
# Loading models 
# To load a model, need to re-create the model structure and load the state dictionary into it 

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [10]:
# Now we can use the model to make some predictions - yay!

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover", 
    "Dress", 
    "Coat", 
    "Sandal", 
    "Shirt", 
    "Sneaker", 
    "Bag", 
    "Ankle Boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle Boot", Actual: "Ankle Boot"
